In [3]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import json
import os

# Configs
img_size = 96
batch_size = 32
initial_epochs = 15
fine_tune_epochs = 5
train_dir = "/Users/srvns/Downloads/images/train"
test_dir = "/Users/srvns/Downloads/images/validation"
output_model_path = "FER_3/expression_model.h5"
label_json_path = "FER_3/class_labels.json"

# Step 1: Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=25,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    shear_range=0.15,
    horizontal_flip=True
)
test_datagen = ImageDataGenerator(rescale=1./255)

train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical'
)
test_data = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical'
)

# Step 2: Save class labels
with open(label_json_path, 'w') as f:
    json.dump(train_data.class_indices, f)

# Step 3: Create model
base_model = MobileNetV2(include_top=False, input_shape=(img_size, img_size, 3), weights='imagenet')
base_model.trainable = False  # Freeze base layers for first phase

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.3)(x)
outputs = Dense(train_data.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=outputs)

model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Step 4: Initial training
print("\n🔧 Training top layers...")
model.fit(train_data, validation_data=test_data, epochs=initial_epochs)

# Step 5: Fine-tune all layers
base_model.trainable = True
model.compile(optimizer=Adam(learning_rate=1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

print("\n Fine-tuning full model...")
model.fit(train_data, validation_data=test_data, epochs=fine_tune_epochs)

# Step 6: Save model
model.save(output_model_path)
print(f"\n✅ Model saved to {output_model_path}")


Found 28821 images belonging to 7 classes.
Found 7066 images belonging to 7 classes.

🔧 Training top layers...
Epoch 1/15


/opt/anaconda3/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


901/901 ━━━━━━━━━━━━━━━━━━━━ 61s 66ms/step - accuracy: 0.2300 - loss: 2.0346 - val_accuracy: 0.3389 - val_loss: 1.6838
Epoch 2/15
901/901 ━━━━━━━━━━━━━━━━━━━━ 71s 79ms/step - accuracy: 0.2962 - loss: 1.7391 - val_accuracy: 0.3823 - val_loss: 1.6133
Epoch 3/15
901/901 ━━━━━━━━━━━━━━━━━━━━ 75s 83ms/step - accuracy: 0.3300 - loss: 1.6814 - val_accuracy: 0.3906 - val_loss: 1.5787
Epoch 4/15
901/901 ━━━━━━━━━━━━━━━━━━━━ 77s 85ms/step - accuracy: 0.3509 - loss: 1.6438 - val_accuracy: 0.3954 - val_loss: 1.5590
Epoch 5/15
901/901 ━━━━━━━━━━━━━━━━━━━━ 78s 86ms/step - accuracy: 0.3480 - loss: 1.6431 - val_accuracy: 0.4063 - val_loss: 1.5473
Epoch 6/15
901/901 ━━━━━━━━━━━━━━━━━━━━ 82s 91ms/step - accuracy: 0.3600 - loss: 1.6224 - val_accuracy: 0.4098 - val_loss: 1.5447
Epoch 7/15
901/901 ━━━━━━━━━━━━━━━━━━━━ 94s 104ms/step - accuracy: 0.3671 - loss: 1.6155 - val_accuracy: 0.4155 - val_loss: 1.5331
Epoch 8/15
901/901 ━━━━━━━━━━━━━━━━━━━━ 102s 113ms/step - accuracy: 0.3723 - loss: 1.5989 - val_accu


✅ Model saved to FER_3/expression_model.h5


In [2]:
# Step 2: Save class labels
os.makedirs(os.path.dirname(label_json_path), exist_ok=True)
with open(label_json_path, 'w') as f:
    json.dump(train_data.class_indices, f)


In [4]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    zoom_range=0.2,
    rotation_range=25,
    shear_range=0.2,
    horizontal_flip=True,
    width_shift_range=0.2,
    height_shift_range=0.2,
    fill_mode='nearest',
    validation_split=0.2
)


In [5]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, Input

base_model = MobileNetV2(weights='imagenet', include_top=False, input_tensor=Input(shape=(96, 96, 3)))
base_model.trainable = False  # Start with frozen base

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.4)(x)
output = Dense(7, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])



/var/folders/c1/ltm2gbn57n98f6kpk_g4qkv80000gn/T/ipykernel_19200/3397637321.py:5: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(weights='imagenet', include_top=False, input_tensor=Input(shape=(96, 96, 3)))


In [6]:
base_model.trainable = True
for layer in base_model.layers[:-40]:  # Freeze all but last 40 layers
    layer.trainable = False

model.compile(optimizer=tf.keras.optimizers.Adam(1e-5), loss='categorical_crossentropy', metrics=['accuracy'])


In [7]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)
]


In [8]:
loss = tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1)


In [11]:
pip install tensorflow opencv-python streamlit matplotlib


Note: you may need to restart the kernel to use updated packages.


In [14]:
import os
import json
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# Configs
img_size = 224
batch_size = 32
train_dir = "/Users/srvns/Downloads/images/train"
val_dir = "/Users/srvns/Downloads/images/validation"
label_json_path = os.path.join(train_dir, "../class_labels.json")

# Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=25,
    zoom_range=0.2,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

# Data loading
train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical'
)

val_data = val_datagen.flow_from_directory(
    val_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical'
)

# Save class labels
with open(label_json_path, 'w') as f:
    json.dump(train_data.class_indices, f)


Found 28821 images belonging to 7 classes.
Found 7066 images belonging to 7 classes.


In [15]:
from tensorflow.keras.optimizers import Adam

# Load base model
base_model = EfficientNetB0(include_top=False, weights='imagenet', input_shape=(img_size, img_size, 3))

# Freeze base model layers initially
base_model.trainable = False

# Add custom classification head
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.4)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.3)(x)
predictions = Dense(train_data.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(learning_rate=1e-3),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_2         │ (None, 224, 224,  │          0 │ input_layer_3[0]… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization_1     │ (None, 224, 224,  │          7 │ rescaling_2[0][0] │
│ (Normalization)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_3         │ (None, 224, 224,  │          0 │ normalization_1[… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv_pad       │ (None, 225, 225,  │          0 │ rescaling_3[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 112, 112,  │        864 │ stem_conv_pad[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 112, 112,  │        128 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 112, 112,  │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_dwconv      │ (None, 112, 112,  │        288 │ stem_activation[… │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_bn          │ (None, 112, 112,  │        128 │ block1a_dwconv[0… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_activation  │ (None, 112, 112,  │          0 │ block1a_bn[0][0]  │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_squeeze  │ (None, 32)        │          0 │ block1a_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reshape  │ (None, 1, 1, 32)  │          0 │ block1a_se_squee… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reduce   │ (None, 1, 1, 8)   │        264 │ block1a_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_expand   │ (None, 1, 1, 32)  │        288 │ block1a_se_reduc… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_excite   │ (None, 112, 112,  │          0 │ block1a_activati… │
│ (Multiply)          │ 32)               │            │ block1a_se_expan… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 112, 112,  │        512 │ block1a_se_excit

 Total params: 4,214,442 (16.08 MB)

 Trainable params: 164,871 (644.03 KB)

 Non-trainable params: 4,049,571 (15.45 MB)

In [17]:
from tensorflow.keras.applications import EfficientNetV2B0
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Define image size and number of classes
img_size = 96  # or 224 if your images are large enough
num_classes = 7

# Load EfficientNetV2B0 base model
base_model = EfficientNetV2B0(
    include_top=False,
    input_tensor=Input(shape=(img_size, img_size, 3)),
    weights='imagenet'
)

# Freeze base initially
base_model.trainable = False

# Custom head
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.4)(x)
output = Dense(num_classes, activation='softmax')(x)

# Build model
model = Model(inputs=base_model.input, outputs=output)

# Compile model
model.compile(optimizer=Adam(learning_rate=1e-3),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


24274472/24274472 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


In [18]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir = "/Users/srvns/Downloads/images/train"
val_dir = "/Users/srvns/Downloads/images/validation"
batch_size = 32
img_size = 96  # should match model input

# Apply augmentations for better generalization
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode="nearest"
)

val_datagen = ImageDataGenerator(rescale=1./255)

# Flow data from directories
train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical'
)

val_data = val_datagen.flow_from_directory(
    val_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical'
)


Found 28821 images belonging to 7 classes.
Found 7066 images belonging to 7 classes.


In [19]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV3Large
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
import json
import os

num_classes = train_data.num_classes
input_shape = (img_size, img_size, 3)

# Base model
base_model = MobileNetV3Large(
    input_shape=input_shape,
    include_top=False,
    weights='imagenet'
)
base_model.trainable = False  # Freeze base

# Add classification head
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.3)(x)
output = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)

model.compile(optimizer=Adam(learning_rate=1e-3),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()


/opt/anaconda3/lib/python3.12/site-packages/keras/src/applications/mobilenet_v3.py:517: UserWarning: `input_shape` is undefined or non-square, or `rows` is not 224. Weights for input shape (224, 224) will be loaded as the default.
  return MobileNetV3(


12683000/12683000 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_5       │ (None, 96, 96, 3) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_5         │ (None, 96, 96, 3) │          0 │ input_layer_5[0]… │
│ (Rescaling)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv (Conv2D)       │ (None, 48, 48,    │        432 │ rescaling_5[0][0] │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv_bn             │ (None, 48, 48,    │         64 │ conv[0][0]        │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 48, 48,    │          0 │ conv_bn[0][0]     │
│ (Activation)        │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 48, 48,    │        144 │ activation[0][0]  │
│ (DepthwiseConv2D)   │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 48, 48,    │         64 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu (ReLU)        │ (None, 48, 48,    │          0 │ expanded_conv_de… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 48, 48,    │        256 │ re_lu[0][0]       │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 48, 48,    │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_add   │ (None, 48, 48,    │          0 │ activation[0][0], │
│ (Add)               │ 16)               │            │ expanded_conv_pr… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_1_ex… │ (None, 48, 48,    │      1,024 │ expanded_conv_ad… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_1_ex… │ (None, 48, 48,    │        256 │ expanded_conv_1_… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_1 (ReLU)      │ (None, 48, 48,    │          0 │ expanded_conv_1_… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_1_de… │ (None, 49, 49,    │          0 │ re_lu_1[0][0]     │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_1_de… │ (None, 24, 24,    │        576 │ expanded_conv_1_… │
│ (DepthwiseConv2D)   │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_1_de… │ (None, 24, 24,    │        256 │ expanded_conv_1_

 Total params: 3,120,263 (11.90 MB)

 Trainable params: 123,911 (484.03 KB)

 Non-trainable params: 2,996,352 (11.43 MB)

In [20]:
# Warm-up training
history_warmup = model.fit(
    train_data,
    validation_data=val_data,
    epochs=5,
    callbacks=[early_stop],
    verbose=1
)


Epoch 1/5
901/901 ━━━━━━━━━━━━━━━━━━━━ 62s 66ms/step - accuracy: 0.2135 - loss: 1.8749 - val_accuracy: 0.2583 - val_loss: 1.8018
Epoch 2/5
901/901 ━━━━━━━━━━━━━━━━━━━━ 64s 71ms/step - accuracy: 0.2472 - loss: 1.8187 - val_accuracy: 0.2583 - val_loss: 1.7995
Epoch 3/5
901/901 ━━━━━━━━━━━━━━━━━━━━ 69s 77ms/step - accuracy: 0.2500 - loss: 1.8053 - val_accuracy: 0.2596 - val_loss: 1.7902
Epoch 4/5
901/901 ━━━━━━━━━━━━━━━━━━━━ 63s 69ms/step - accuracy: 0.2530 - loss: 1.7998 - val_accuracy: 0.2617 - val_loss: 1.7836
Epoch 5/5
901/901 ━━━━━━━━━━━━━━━━━━━━ 71s 79ms/step - accuracy: 0.2554 - loss: 1.7973 - val_accuracy: 0.2648 - val_loss: 1.7797
Restoring model weights from the end of the best epoch: 5.


In [23]:
# Unfreeze base model
base_model.trainable = True

# Compile again with a very low learning rate
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Fine-tune training
history_finetune = model.fit(
    train_data,
    validation_data=val_data,
    epochs=5,
    callbacks=[early_stop],
    verbose=1
)


Epoch 1/5
901/901 ━━━━━━━━━━━━━━━━━━━━ 276s 286ms/step - accuracy: 0.2544 - loss: 2.2706 - val_accuracy: 0.1656 - val_loss: 2.0202
Epoch 2/5
901/901 ━━━━━━━━━━━━━━━━━━━━ 293s 325ms/step - accuracy: 0.2882 - loss: 1.8658 - val_accuracy: 0.1609 - val_loss: 1.9520
Epoch 3/5
901/901 ━━━━━━━━━━━━━━━━━━━━ 254s 282ms/step - accuracy: 0.3150 - loss: 1.7457 - val_accuracy: 0.2024 - val_loss: 1.8497
Epoch 4/5
901/901 ━━━━━━━━━━━━━━━━━━━━ 265s 294ms/step - accuracy: 0.3488 - loss: 1.6536 - val_accuracy: 0.2571 - val_loss: 1.7865
Epoch 5/5
901/901 ━━━━━━━━━━━━━━━━━━━━ 228s 253ms/step - accuracy: 0.3724 - loss: 1.6125 - val_accuracy: 0.3651 - val_loss: 1.6402
Restoring model weights from the end of the best epoch: 5.


In [24]:
# Save the fine-tuned model
model.save("expression_model_finetuned.h5")

# Save class labels
import json
with open("class_labels.json", "w") as f:
    json.dump(train_data.class_indices, f)
